In [2]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"
import re

In [3]:
device = "cpu"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

In [5]:
def generate_subtasks(task_description, model, tokenizer, num_subtasks):
    prompt = (
        f"You are a task planner. Break down the following task into exactly {num_subtasks} clear and actionable steps. "
        "Each subtask should be practical, specific, and easy to follow. The subtasks should be ordered logically, "
        "and should focus on accomplishing the task in a methodical way. Avoid any filler, general explanations, or placeholders. "
        "The goal is for someone to be able to follow these steps and complete "
        "the task without needing further clarification.\n\n"
        f"Task: {task_description}\n\n"
        "Subtasks:"
    )

    tokenizer.pad_token = tokenizer.eos_token

    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
        
        outputs = model.generate(
            **inputs,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2,
            num_beams=1,
            early_stopping=True,
            max_new_tokens=256
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    subtasks_section = generated_text.split("Subtasks:", 1)[-1].strip()
    subtasks = []
    
    # Match valid subtasks that start with a number followed by a period and space
    valid_subtask_regex = r"^\d+\.\s+"

    for line in subtasks_section.split("\n"):
        line = line.strip()
        if line and re.match(valid_subtask_regex, line):
            subtask_text = re.sub(r"^\d+\.\s*", "", line)  # Remove numbering
            subtasks.append(subtask_text.strip())  # Add the subtask text

    # Ensure exactly 'num_subtasks' subtasks, truncating if necessary
    subtasks = subtasks[:num_subtasks]

    # If there are fewer subtasks, fill in placeholders but avoid adding placeholders if it is an incomplete task
    while len(subtasks) < num_subtasks:
        subtasks.append("Complete the task.")

    return subtasks

In [6]:
subtasks = generate_subtasks("plan wedding", model, tokenizer, num_subtasks=8)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [7]:
for i, subtask in enumerate(subtasks, 1):
    print(f"{i}. {subtask}")

1. Decide on the date of the wedding.
2. Choose a venue.
3. Create a guest list. Determine how many guests you can invite to your special day. Keep in mind that you'll need to accommodate guests with disabilities, as well as those with special dietary needs.
4. Plan the menu. You'll want to decide on a menu that will satisfy your guests' tastes and dietary restrictions. Research local restaurants and consider factors like price, location, availability, quality, service, food quality and price. Also, consider the dietary requirements of your guest. For example, if you have guests who are gluten-free, you may need a gluten free menu or a substitute for gluten. If you're planning a vegetarian or vegan wedding, research restaurants that offer vegetarian and vegan options. Finally, decide whether you want a sit-down dinner, a buffet, appetizers, main course, dessert, wine, beer, cocktails,
5. Placeholder.
6. Placeholder.
7. Placeholder.
8. Placeholder.
